# Installing dependencies

In [1]:
!pip install kagglehub
!pip install pyspark
!wget https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.2.0/hadoop-aws-3.2.0.jar
!wget https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.11.375/aws-java-sdk-bundle-1.11.375.jar


In [64]:
# for deleting files from colab
# import shutil

# shutil.rmtree('./airline_delay_ranking.csv')

# Login to Kaggle

In [2]:
import kagglehub

kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


## Downloading datasets from Kaggle

In [3]:
path_2009 = kagglehub.dataset_download("yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018")

print("Path to dataset files:", path_2009)
path_2019 = kagglehub.dataset_download("patrickzel/flight-delay-and-cancellation-dataset-2019-2023")

print("Path to dataset files:", path_2019)

100%|██████████| 1.95G/1.95G [00:18<00:00, 113MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018/versions/1


100%|██████████| 140M/140M [00:01<00:00, 124MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/patrickzel/flight-delay-and-cancellation-dataset-2019-2023/versions/7


Listing the files that have been collected from kaggle.

In [6]:
import os

files_2009 = os.listdir(path_2009)
print("first files downloaded:", files_2009)
files_2019 = os.listdir(path_2019)
print("second Files downloaded:", files_2019)

first files downloaded: ['2013.csv', '2014.csv', '2010.csv', '2016.csv', '2015.csv', '2009.csv', '2012.csv', '2018.csv', '2011.csv', '2017.csv']
second Files downloaded: ['flights_sample_3m.csv', 'dictionary.html']


Creating a mapping to change the column names because they differ between the 3 kaggle datasets. Changing all to fit the original airline.csv

In [18]:
# Mapping for converting columns in the second and third indexes to the first index
column_mapping = {
    # Second Index to First Index
    'FL_DATE': 'Date',
    'OP_CARRIER': 'UniqueCarrier',
    'OP_CARRIER_FL_NUM': 'FlightNum',
    'ORIGIN': 'Origin',
    'DEST': 'Dest',
    'CRS_DEP_TIME': 'CRSDepTime',
    'DEP_TIME': 'DepTime',
    'DEP_DELAY': 'DepDelay',
    'TAXI_OUT': 'TaxiOut',
    'WHEELS_OFF': 'WheelsOff',
    'WHEELS_ON': 'WheelsOn',
    'TAXI_IN': 'TaxiIn',
    'CRS_ARR_TIME': 'CRSArrTime',
    'ARR_TIME': 'ArrTime',
    'ARR_DELAY': 'ArrDelay',
    'CANCELLED': 'Cancelled',
    'CANCELLATION_CODE': 'CancellationCode',
    'DIVERTED': 'Diverted',
    'CRS_ELAPSED_TIME': 'CRSElapsedTime',
    'ACTUAL_ELAPSED_TIME': 'ActualElapsedTime',
    'AIR_TIME': 'AirTime',
    'DISTANCE': 'Distance',
    'CARRIER_DELAY': 'CarrierDelay',
    'WEATHER_DELAY': 'WeatherDelay',
    'NAS_DELAY': 'NASDelay',
    'SECURITY_DELAY': 'SecurityDelay',
    'LATE_AIRCRAFT_DELAY': 'LateAircraftDelay',

    # Third Index to First Index
    'AIRLINE_CODE': 'UniqueCarrier',  # assuming similar mapping if applicable
    'FL_NUMBER': 'FlightNum',
}

## Using SPARK to process the data
Here we are loading the csv files as spark dataframes to save some space and compute.

In [31]:
from pyspark.sql import SparkSession

# create Spark session
spark = SparkSession.builder.appName("airline_processing").getOrCreate()

def load_and_rename(file_path, column_mapping):
    # Load CSV directly into Spark DataFrame
    spark_df = spark.read.csv(file_path, header=True, inferSchema=True)

    # Remove unnamed columns (any column name containing "Unnamed")
    unnamed_columns = [col for col in spark_df.columns if "Unnamed" in col]
    spark_df = spark_df.drop(*unnamed_columns)

    # Rename columns based on mapping
    for old_col, new_col in column_mapping.items():
        if old_col in spark_df.columns:
            spark_df = spark_df.withColumnRenamed(old_col, new_col)
    return spark_df

In [32]:
import pandas as pd
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("ProcessLargeCSVInChunks") \
    .getOrCreate()

# Define your large CSV file path
csv_file_path = "airline.csv"

# Define the output directory for intermediate Parquet files
intermediate_path = "airline_parquet/"

# Parameters for chunk processing
chunk_size = 100000  # Number of rows per chunk

# Process CSV in chunks using Pandas
chunk_number = 0
for chunk in pd.read_csv(csv_file_path, chunksize=chunk_size):
    # # Rename columns
    # chunk.rename(columns=column_mapping, inplace=True)

    # Convert Pandas DataFrame to Spark DataFrame
    spark_df = spark.createDataFrame(chunk)

    # Save each chunk as a separate Parquet file
    spark_df.write.parquet(f"{intermediate_path}/airline_file_{chunk_number}.parquet")

    # Free up memory
    del spark_df
    chunk_number += 1


In [33]:
chunk_number

99

In [34]:
# process files
intermediate_path = "airline_parquet/"
sparks_dfs = []
# i = 0
# spark_df = load_and_rename("airline.csv", column_mapping)
# spark_df.write.parquet(f"{intermediate_path}/airline_file_{i}.parquet")
# i += 1

for files in files_2009:
    path = path_2009 + '/' + files
    spark_df = load_and_rename(path, column_mapping)
    spark_df.write.parquet(f"{intermediate_path}/airline_file_{chunk_number}.parquet")
    chunk_number += 1

spark_df = load_and_rename(path_2019 + r'/flights_sample_3m.csv', column_mapping)
spark_df.write.parquet(f"{intermediate_path}/airline_file_{chunk_number}.parquet")

In [35]:
chunk_number

109

We are correcting some datatype differences here due to having taken info from 3 different sources.

In [37]:
from pyspark.sql.types import IntegerType, LongType, DoubleType
from pyspark.sql.functions import col

combined_df = spark.read.parquet("./airline_parquet/*.parquet")

# Define schema corrections
schema_corrections = {
    "CRSArrTime": LongType(),
    "CRSDepTime": LongType(),
    "FlightNum": IntegerType(),
    "Cancelled": DoubleType()
    # Add more columns and their desired types if needed
}

# Apply schema corrections
for column_name, desired_type in schema_corrections.items():
    combined_df = combined_df.withColumn(column_name, col(column_name).cast(desired_type))

combined_df.printSchema()

root
 |-- ActualElapsedTime: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- ArrDelay: double (nullable = true)
 |-- ArrTime: double (nullable = true)
 |-- CRSArrTime: long (nullable = true)
 |-- CRSDepTime: long (nullable = true)
 |-- CRSElapsedTime: double (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Cancelled: double (nullable = true)
 |-- CarrierDelay: double (nullable = true)
 |-- DayOfWeek: long (nullable = true)
 |-- DayofMonth: long (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Diverted: long (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- LateAircraftDelay: double (nullable = true)
 |-- Month: long (nullable = true)
 |-- NASDelay: double (nullable = true)
 |-- Origin: string (nullable = true)
 |-- SecurityDelay: double (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- Taxi

## Processing Spark dataframe to get Airline Ranking by Number of Delays

In [57]:
from pyspark.sql import functions as F

# Retrieve Delay rankings
delayed_airlines = combined_df.withColumn(
    "total_delay",
    F.when(
        (F.col("CarrierDelay") > 0) |
        (F.col("WeatherDelay") > 0) |
        (F.col("NASDelay") > 0) |
        (F.col("SecurityDelay") > 0) |
        (F.col("LateAircraftDelay") > 0),
        1
    ).otherwise(0)
)

result = delayed_airlines.groupBy("UniqueCarrier").agg(
    F.sum("total_delay").alias("total_delay")  # Assuming "Description" column exists in carrier_df
).orderBy(F.desc("total_delay"))

# Show the result
row_count = result.count()
result.show(row_count)
# result.show()

+-------------+-----------+
|UniqueCarrier|total_delay|
+-------------+-----------+
|           WN|    3276267|
|           DL|    2324107|
|           AA|    2307203|
|           UA|    1813945|
|           US|    1441661|
|           OO|    1207980|
|           EV|    1069376|
|           MQ|     828319|
|           NW|     745249|
|           CO|     710266|
|           B6|     608320|
|           AS|     408640|
|           TW|     300220|
|           XE|     294301|
|           HP|     258771|
|           FL|     248637|
|           F9|     219183|
|           YV|     211043|
|           9E|     206959|
|           OH|     167263|
|           NK|     128747|
|           VX|      79576|
|           EA|      73673|
|           PI|      69978|
|           HA|      69388|
|           YX|      56901|
|       PA (1)|      25440|
|           G4|      21231|
|           DH|      19351|
|           PS|       6639|
|           AQ|       6292|
|       ML (1)|       5737|
|           TZ|     

Saving the results into a zip file.

In [65]:
result.coalesce(1).write.parquet("airline_delay_ranking.parquet")

In [68]:
!zip -r airline_delay_ranking.zip airline_delay_ranking.parquet

  adding: airline_delay_ranking.parquet/ (stored 0%)
  adding: airline_delay_ranking.parquet/_SUCCESS (stored 0%)
  adding: airline_delay_ranking.parquet/.part-00000-57a108a2-fefc-443f-925f-e0dd4a6d9d27-c000.snappy.parquet.crc (stored 0%)
  adding: airline_delay_ranking.parquet/._SUCCESS.crc (stored 0%)
  adding: airline_delay_ranking.parquet/part-00000-57a108a2-fefc-443f-925f-e0dd4a6d9d27-c000.snappy.parquet (deflated 30%)


## Saving all Spark data and parquet files to S3

In [69]:
from dotenv import load_dotenv
import boto3
from botocore.exceptions import NoCredentialsError
import os

load_dotenv()
# Define AWS credentials and initialize S3 client
s3 = boto3.client(
    's3',
    aws_access_key_id=os.environ.get("AWS_ACCESS_KEY"),
    aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY"),
    region_name='us-east-1'  # Set your region
)

# Define the file to upload and target S3 bucket details
local_file_path = './airline_delay_ranking.zip'
bucket_name = 'is459-project-output-data'
s3_file_path = 'airline_delay_ranking.zip'

try:
    # Upload the file
    s3.upload_file(local_file_path, bucket_name, s3_file_path)
    print(f"File uploaded successfully to s3://{bucket_name}/{s3_file_path}")
except FileNotFoundError:
    print("The file was not found")
except NoCredentialsError:
    print("Credentials not available")


File uploaded successfully to s3://is459-project-output-data/airline_delay_ranking.zip


In [73]:
# save combined spark dataframes

# Define local directory and S3 bucket details
local_directory = './airline_parquet'  # Replace with your directory path
bucket_name = 'is459-project-output-data'  # Replace with your S3 bucket name
s3_folder = 'kaggle'  # Replace with the desired folder path in S3

# Function to upload files in a directory to S3
def upload_files_to_s3(local_path, s3_path):
    for root, dirs, files in os.walk(local_path):
        for file in files:
            # Only upload files with .parquet extension
            if file.endswith(".parquet"):
                local_file_path = os.path.join(root, file)
                s3_file_path = os.path.join(s3_path, os.path.relpath(local_file_path, local_path))

                try:
                    # Upload the file to S3
                    s3.upload_file(local_file_path, bucket_name, s3_file_path)
                    print(f"Uploaded {local_file_path} to s3://{bucket_name}/{s3_file_path}")
                except FileNotFoundError:
                    print(f"The file {file} was not found")
                except NoCredentialsError:
                    print("Credentials not available")

# Upload all Parquet files from local directory to S3
upload_files_to_s3(local_directory, s3_folder)

Uploaded ./airline_parquet/airline_file_93.parquet/part-00001-d4574420-730f-4efa-a27e-bcaa9968623d-c000.snappy.parquet to s3://is459-project-output-data/kaggle/airline_file_93.parquet/part-00001-d4574420-730f-4efa-a27e-bcaa9968623d-c000.snappy.parquet
Uploaded ./airline_parquet/airline_file_93.parquet/part-00000-d4574420-730f-4efa-a27e-bcaa9968623d-c000.snappy.parquet to s3://is459-project-output-data/kaggle/airline_file_93.parquet/part-00000-d4574420-730f-4efa-a27e-bcaa9968623d-c000.snappy.parquet
Uploaded ./airline_parquet/airline_file_41.parquet/part-00001-c662b2bc-a419-4869-b552-c43ea2bfde6b-c000.snappy.parquet to s3://is459-project-output-data/kaggle/airline_file_41.parquet/part-00001-c662b2bc-a419-4869-b552-c43ea2bfde6b-c000.snappy.parquet
Uploaded ./airline_parquet/airline_file_41.parquet/part-00000-c662b2bc-a419-4869-b552-c43ea2bfde6b-c000.snappy.parquet to s3://is459-project-output-data/kaggle/airline_file_41.parquet/part-00000-c662b2bc-a419-4869-b552-c43ea2bfde6b-c000.snappy.